In [1]:
### 가상환경 : test, python 3.7.16
# crawl setting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 55.3MB/s]


In [9]:
# 올리브영 > 토너/로션/올인원
### 모든 상품의 goodsno 가져오기
goodsno = list()

base_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx={}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat100000100010008_Small"
for i in range(0,16):
    url = base_url.format(i+1)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser', from_encoding = 'utf-8')
    if i != 15: #마지막 페이지가 아니라면
        for j in range(0,12): # 4 * 12 = 48개 상품에 대해 goodsno 크롤링
            temp = soup.find('div', id='Contents').find_all('ul', 'cate_prd_list gtm_cate_list')[j]
            no = temp.find_all('div', "prd_info")
            for m in range(0,4):
                goodsno.append(no[m].find('a')['data-ref-goodsno'])
    else: #마지막 페이지라면
        for j in range(0,9): # 4*9-1 = 35개 상품에 대해 goodsno 크롤링
            temp = soup.find('div', id='Contents').find_all('ul', 'cate_prd_list gtm_cate_list')[j]
            no = temp.find_all('div', "prd_info")
            if j != 8:
                for m in range(0,4):
                    goodsno.append(no[m].find('a')['data-ref-goodsno'])
            else: #마지막 페이지의 마지막 줄에는 상품 3개만 존재
                for m in range(0,3):
                    goodsno.append(no[m].find('a')['data-ref-goodsno'])
    print(i,"번째 페이지 goodsno 스크래핑 완료")

0 번째 페이지 goodsno 스크래핑 완료
1 번째 페이지 goodsno 스크래핑 완료
2 번째 페이지 goodsno 스크래핑 완료
3 번째 페이지 goodsno 스크래핑 완료
4 번째 페이지 goodsno 스크래핑 완료
5 번째 페이지 goodsno 스크래핑 완료
6 번째 페이지 goodsno 스크래핑 완료
7 번째 페이지 goodsno 스크래핑 완료
8 번째 페이지 goodsno 스크래핑 완료
9 번째 페이지 goodsno 스크래핑 완료
10 번째 페이지 goodsno 스크래핑 완료
11 번째 페이지 goodsno 스크래핑 완료
12 번째 페이지 goodsno 스크래핑 완료
13 번째 페이지 goodsno 스크래핑 완료
14 번째 페이지 goodsno 스크래핑 완료
15 번째 페이지 goodsno 스크래핑 완료


In [11]:
# 개별 상품 url 리스트로 저장
url = list()
for i in range(len(goodsno)): #url 리스트로 저장
    num = goodsno[i]
    globals()['url_{}'.format(i)] = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode'.format(num)
    url.append(globals()['url_{}'.format(i)])

In [60]:
#goodsno, url 저장
import pandas as pd
import pickle

goods = pd.DataFrame({"goodsno":goodsno, "url":url}) #데이터프레임으로 변경
goods.head()

,goodsno,url
0,A000000006564,https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000006565,https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000137180,https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000184129,https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000170266,https://www.oliveyoung.co.kr/store/goods/getGo...


In [61]:
import numpy as np
#상품 정보 크롤링 : 상품명, 카테고리, 브랜드, 가격, 용량, 주요사양, 성분
goods['상품명'] = np.nan; goods['카테고리'] = np.nan; goods['브랜드'] = np.nan; goods['가격'] = np.nan
goods['용량'] = np.nan; goods['주요사양'] = np.nan; goods['성분'] = np.nan

#상품 리뷰 크롤링 : 평점, 평점 분포, 피부타입, 피부타입별 %, 피부고민, 피부고민별 %, 자극도, 자극도별 %
## 전체 평점 외에는 반복문 내에서 정의
goods['전체 평점'] = np.nan

goods.head()

,goodsno,url,상품명,카테고리,브랜드,가격,용량,주요사양,성분,전체 평점
0,A000000006564,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A000000006565,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A000000137180,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A000000184129,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A000000170266,https://www.oliveyoung.co.kr/store/goods/getGo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#개별 상품 페이지 크롤링
driver = webdriver.Chrome(service=service, options=chrome_options)

for i,j in enumerate(goods.loc[:, 'url']): #나눠서 돌리기
    if pd.isnull(goods.loc[i, "용량"]): #크롤링 안 된 부분부터
        resp = requests.get(j)
        if resp.status_code == requests.codes.ok: #status_code == 200인 경우만
            driver.get(j) #개별 상품 페이지로 이동
            
            seed = np.random.randint(100)
            np.random.seed(seed)
            a = np.random.randint(5)
            time.sleep(a) #생성한 난수만큼 sleep

            item = driver.find_element(By.CLASS_NAME, 'prd_name').text 
            goods.loc[i, '상품명'] = item
            brand = driver.find_element(By.CLASS_NAME, "prd_brand").text
            goods.loc[i, '브랜드'] = brand
            category = driver.find_elements(By.CLASS_NAME, "cate_y")[-1].text
            goods.loc[i, '카테고리'] = category #카테고리 : 마지막 카테고리만 가져오기
            price = driver.find_element(By.CLASS_NAME, "price-2").text 
            goods.loc[i, '가격'] = price #가격

            info = "var element = document.querySelector('.goods_buyinfo');element.click();"
            driver.execute_script(info) #구매 정보 클릭
            driver.implicitly_wait(20)
            amount = driver.find_element(By.CSS_SELECTOR, "#artcInfo > dl:nth-child(2) > dd").text
            goods.loc[i, '용량'] = amount
            spec = driver.find_element(By.CSS_SELECTOR, "#artcInfo > dl:nth-child(3) > dd").text 
            goods.loc[i, '주요사양'] = spec
            contain = driver.find_element(By.CSS_SELECTOR, "#artcInfo > dl:nth-child(8) > dd").text
            goods.loc[i, '성분'] = contain

            review = "var element = document.querySelector('.goods_reputation');element.click();"
            driver.execute_script(review) #리뷰 클릭
            driver.implicitly_wait(10)
            try:
                total_rate = driver.find_element(By.CSS_SELECTOR, "#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong").text
                goods.loc[i, '전체 평점'] = total_rate
                for m in range(1,6):
                    selector = "#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.graph_area > ul > li:nth-child({}) > span.per"
                    rate = driver.find_element(By.CSS_SELECTOR, selector.format(m)).text
                    col = f"{6-m}점"
                    goods.loc[i, col] = rate
            except NoSuchElementException: #리뷰가 없는 경우도 있다
                print("리뷰 없음")

            try:
                for k in range(1,4):
                    for l in range(1,4): #건성(1,1)-복합성(1,2)-지성(1,3), 보습(2,1)-진정(2,2)-주름/미백(2,3), 순함(3,1)-보통(3,2)-자극(3,3) 순
                        selector = "#gdasContentsArea > div > div.poll_all.clrfix > dl:nth-child({}) > dd > ul > li:nth-child({}) > em"
                        skin = driver.find_element(By.CSS_SELECTOR, selector.format(k,l)).text
                        col = f"{k}_{l}피부"
                        goods.loc[i, col] = skin
            except NoSuchElementException: #세부사항이 없는 경우도 있다
                print("세부사항 없음")
        else:
            break

In [63]:
with open("goods.pickle", "wb") as g:
    pickle.dump(goods, g) #크롤링 결과 저장